In [1]:
def get_group_place(df, matchId) :
    # 각 match 선택
    match_data = df[df['matchId'].values == matchId]
    num = match_data['maxPlace'].mean()
    
    if match_data['groupId'].nunique() == 1 : 
        gid = match_data['groupId'].values[0]
        final_dictionary.update({gid : 0})
        return 
    
    elif match_data['groupId'].nunique() == 2 :
        gids = match_data['groupId'].unique()
        g1 = gids[0]
        g2 = gids[1]
        g1_kills = match_data[match_data['groupId'].values == g1]['kills'].sum()
        if g1_kills == len(match_data[match_data['groupId'].values == g2]) : 
            final_dictionary.update({g1 : 1,
                                     g2 : 0})
        else : 
            final_dictionary.update({g1 : 0,
                                     g2 : 1})
        return 
        
    # solo or not 
    mtype = match_data['matchType'].iloc[0]
    
    # empty  
    Maxplace = match_data.iloc[0]['maxPlace']
    Numgroups = match_data.iloc[0]['numGroups']
    t = Maxplace - Numgroups
    pivot = int(np.round((Maxplace - t)/(t+1)))
    indices = []
    for i in range(1, t+1) : 
        indices.append(i*pivot)
    
    # kill별로 sort
    match_data = match_data.sort_values(by = ['kills', 'killPlace'])
    match_data = match_data[['kills', 'groupId', 'killPlace']]

    # 해당 kill에 해당하는 user가 1명인 경우 제외   
    kills_values = match_data['kills'].value_counts()
    kills_unique = kills_values[kills_values.values == 1].index.tolist()
    kills_unique_user = match_data[match_data['kills'].isin(kills_unique)]
    
    kill_type = kills_values.index
    
    ###
    if (match_data['kills'].value_counts() == 1).all() : 
        rank_dict_lst = match_data.groupby('groupId')['kills'].max().sort_values(ascending = False).index.tolist()
    
    else : 
        if 'solo' not in mtype : 
            match_data = match_data[~match_data['kills'].isin(kills_unique)]
            match_data = match_data.sort_values(by = ['kills', 'killPlace'], ascending = True)

        if 'solo' in mtype :
            group_lst = []
            kill_type = sorted(kill_type, reverse = True)

            for kt in kill_type :
                kill_data = match_data[match_data['kills'] == kt]
                kill_data = kill_data.sort_values(by = 'killPlace')
                group_lst =  group_lst + list(kill_data['groupId']) 


        kill_0 = match_data[match_data['kills'] == kill_type[0]][['killPlace', 'groupId']]
        kill_0 = kill_0.drop_duplicates(['groupId'])
        kill_0 = kill_0['groupId'].values

        loc_dict = {}
        rank_dict = {}    
        
        cetain_lst = []
        for i in range(len(kill_0)) : 
            groupId = kill_0[i]
            rank_dict[i] = [groupId]
            loc_dict[groupId] = i

        for kt in kill_type[1:] :
            kill_1 = match_data[match_data['kills'] == kt][['killPlace', 'groupId']]
            kill_1 = kill_1.drop_duplicates(['groupId'])
            kill_1 = kill_1['groupId'].values

            for i in range(len(kill_1)) : 
                cur_group = kill_1[i]
                if cur_group in loc_dict : 
                    continue 
                else : 
                    # upper bound 
                    refer_lst = kill_1[i:]
                    check_lst = [1 if r in loc_dict else 0 for r in refer_lst]
                    if any(check_lst) : 
                        r_index = check_lst.index(1)
                        r = loc_dict[refer_lst[r_index]]
                    else : 
                        for key, values in rank_dict.items() : 
                            if len(values) == 0 : 
                                r = key 
                                break
                            else :
                                r = Numgroups//2

                    # lower bound 
                    refer_lst = kill_1[:i]
                    check_lst = [1 if r in loc_dict else 0 for r in refer_lst]
                    if any(check_lst) : 
                        l_index = max([i for i in range(len(check_lst)) if check_lst[i] == 1])
                        l = loc_dict[refer_lst[l_index]]
                    else : 
                        l = 0

                    if (l+r)/2 in rank_dict : 
                        rank_dict[(l+r)/2].append(cur_group)
                    else : 
                        rank_dict[(l+r)/2] = [cur_group]
                    loc_dict[cur_group] = (l+r)/2


        rank_dict_lst = sorted(rank_dict.items())
        rank_dict_lst = [x[1] for x in rank_dict_lst]
        rank_dict_lst = sum(rank_dict_lst, [])

        for ku in kills_unique : 
            group_name = kills_unique_user[kills_unique_user['kills'] == ku]['groupId'].values[0]
            if group_name not in rank_dict_lst : 
                rank_dict_lst = [group_name] + rank_dict_lst
    rank_dict_lst_df = pd.DataFrame(rank_dict_lst)
    
    winPlace = np.linspace(0, 1, num = int(num), endpoint = True).tolist()
    winPlace = list(reversed(winPlace))
    remove_lst = [] 
    for idx in indices :
        remove_lst.append(winPlace[idx])
        
    for val in remove_lst : 
        if val in winPlace : 
            winPlace.remove(val)
        
    rank_dict_lst_df['pred_winPlace'] = pd.Series(winPlace)
    
    pred_dict = rank_dict_lst_df.set_index([0]).to_dict()['pred_winPlace']
    final_dictionary.update(pred_dict)
    return 